**DATA PROCESSING**

In [1]:
#all libraries
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn import model_selection
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import confusion_matrix
import sklearn.datasets as datasets
from sklearn.metrics import classification_report as report
from sklearn.datasets import make_classification
from numpy import argmax
from sklearn import metrics
from imblearn.over_sampling import SVMSMOTE

In [2]:
#readings data and labelling colums
df = pd.read_csv("data.csv",names=["Chemical_formula","A","B","In_literature","Valence_A","Valence_B","Radius_A","Radius_B","Lowest_distortion","Formation_energy","Stability","Magnetic_moment","Volume_per_atom","Band_gap","a","b","c","alpha","beta","gamma","Vacancy_energy"])

In [3]:
#removing faulty data and replacing strings with a esier convetion
df.drop(df.index[(df["Valence_B"] == "5")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_A"] == "5")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_B"] == "4")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_A"] == "4")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_A"] == "element not in BV")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_B"] == "element not in BV")],axis=0,inplace=True)
df.drop(df.index[(df["Vacancy_energy"] =="-")],axis=0,inplace=True)
df['Valence_A'] = df['Valence_A'].replace(['not balanced'], '0') 
df['Valence_A'] = df['Valence_A'].replace(['3'], '1') 
df['Valence_B'] = df['Valence_B'].replace(['not balanced'], '0') 
df['Valence_B'] = df['Valence_B'].replace(['3'], '1') 
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['cubic'], '0') 
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['orthorhombic'], '1')
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['rhombohedral'], '2')
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['tetragonal'], '3')
df['Magnetic_moment'] = df['Magnetic_moment'].replace(['-'], '0.00')
df['Vacancy_energy'] = df['Vacancy_energy'].replace(['-'], '0.00')
df['In_literature'] = df['In_literature'].replace(["FALSE"], '0') 
df['In_literature'] = df['In_literature'].replace(["TRUE"], '1')

In [4]:
df[1:]#printing data

,Chemical_formula,A,B,In_literature,Valence_A,Valence_B,Radius_A,Radius_B,Lowest_distortion,Formation_energy,...,Magnetic_moment,Volume_per_atom,Band_gap,a,b,c,alpha,beta,gamma,Vacancy_energy
75,Ag2O3,Ag,Ag,0,0,0,1.28,0.95,0,0.351,...,0.00,12.997,0,4.02,4.02,4.02,90,90,90,-2.694
76,AgAlO3,Ag,Al,0,0,0,1.28,0.54,1,-1.51,...,0.00,9.898,0,5.094,5.323,7.3,90,90,90,-1.341
79,AgBO3,Ag,B,0,1,1,0.75,0.27,0,-0.35,...,0.00,8.138,0,3.439,3.439,3.439,90,90,90,-2.641
80,AgBaO3,Ag,Ba,0,0,0,1.28,1.35,1,-0.989,...,0.00,15.631,0.247,6.636,6.739,6.99,90,90,90,-6.975
81,AgBeO3,Ag,Be,0,0,0,1.28,0.45,0,-0.616,...,0.00,8.781,0,3.528,3.528,3.528,90,90,90,-3.058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5322,ZrTmO3,Zr,Tm,0,0,0,0.89,0.96,0,-2.315,...,0.2,14.496,0,4.169,4.169,4.169,90,90,90,-3.645
5326,ZrYO3,Zr,Y,0,0,0,0.89,0.9,0,-2.126,...,0.2,15.277,0,4.243,4.243,4.243,90,90,90,-4.92
5327,ZrYbO3,Zr,Yb,0,0,0,0.89,0.95,1,-3.455,...,0,13.136,4.007,5.558,5.726,8.254,90,90,90,-6.177
5328,ZrZnO3,Zr,Zn,0,0,0,0.89,0.74,0,-1.63,...,0.001,10.804,0,3.78,3.78,3.78,90,90,90,-0.762


In [5]:
#all columns that will be included as a feature
features=["Formation_energy","Stability","Magnetic_moment","Volume_per_atom","Band_gap","Vacancy_energy","Radius_A","Radius_B","Valence_A","Valence_B"]
X = df.loc[1:, features].values
Y = df.loc[1:, "Lowest_distortion"].values

In [6]:
X

array([['0.351', '0.576', '0.00', ..., '0.95', '0', '0'],
       ['-1.51', '0.235', '0.00', ..., '0.54', '0', '0'],
       ['-0.35', '1.074', '0.00', ..., '0.27', '1', '1'],
       ...,
       ['-3.455', '0.205', '0', ..., '0.95', '0', '0'],
       ['-1.63', '1.21', '0.001', ..., '0.74', '0', '0'],
       ['-2.382', '0.906', '0.382', ..., '0.72', '0', '0']], dtype=object)

In [7]:
#spliting data into a test and train set with ratio 0.2
X_train, X_test, Y_train, Y_test = model_selection.train_test_split (X, Y, test_size=0.2, random_state=42)

**OVERSAMPLING**

In [8]:
counter = Counter(Y_train)
for k,v in counter.items():
    per = v / len(Y_train) * 100
    print('Class=%s, n=%s (%.3f%%)' % (k, v, per))

Class=0, n=1283 (60.777%)
Class=2, n=99 (4.690%)
Class=1, n=686 (32.496%)
Class=3, n=43 (2.037%)


In [9]:
oversample = SVMSMOTE()
X_train, Y_train = oversample.fit_resample(X_train, Y_train)

In [10]:
counter = Counter(Y_train)
for k,v in counter.items():
    per = v / len(Y_train) * 100
    print('Class=%s, n=%s (%.3f%%)' % (k, v, per))

Class=0, n=1283 (25.000%)
Class=2, n=1283 (25.000%)
Class=1, n=1283 (25.000%)
Class=3, n=1283 (25.000%)


**FUNCTIONS GLOBAL VARIABLES**

In [11]:
def CRandCM(method,X_test,Y_test):
    Y_pred=method.predict(X_test)
    cr = report(Y_test,  Y_pred)
    cm = confusion_matrix(Y_test, Y_pred)
    print(cr)
    print(cm)

In [12]:
#setting weights so that points which exist in literature has more weight(5), does not exsist in literature has lower weight(1)
#and points produced by SMOTE has least weight(0.5),these weights are hyper parameter,which i have not yet tuned
weights=[]
for i in df["In_literature"]:
    if i=='0':
        weights=np.append(weights,np.array([1]))
    if i=='1':
        weights=np.append(weights,np.array([5]))
    else:
        pass
diff=len(Y_train)-len(weights)
for i in range(diff):
    weights=np.append(weights,np.array([0.5]))
weights.shape

(5132,)

**SUPPORT VECTOR MACHINE-LINEAR KERNEL**

In [13]:
#without weights
SVM_linear = svm.SVC(kernel='linear', C=250,gamma=0.01,random_state=31).fit(X_train, Y_train)
print("Accuracy:",SVM_linear.score(X_test, Y_test))

Accuracy: 0.9034090909090909


In [14]:
CRandCM(SVM_linear,X_test,Y_test)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       331
           1       0.92      0.84      0.88       160
           2       0.41      0.59      0.48        27
           3       0.35      0.70      0.47        10

    accuracy                           0.90       528
   macro avg       0.67      0.77      0.70       528
weighted avg       0.93      0.90      0.91       528

[[320   2   2   7]
 [  2 134  19   5]
 [  1   9  16   1]
 [  0   1   2   7]]


In [15]:
#weighted
SVM_linear2 = svm.SVC(kernel='linear',C=250,gamma=0.01)
SVM_linear2.fit(X_train, Y_train, sample_weight=weights)
print("Accuracy:",SVM_linear2.score(X_test,Y_test))

Accuracy: 0.9090909090909091


In [16]:
CRandCM(SVM_linear2,X_test,Y_test)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       331
           1       0.88      0.88      0.88       160
           2       0.45      0.48      0.46        27
           3       0.40      0.60      0.48        10

    accuracy                           0.91       528
   macro avg       0.68      0.73      0.70       528
weighted avg       0.92      0.91      0.91       528

[[321   4   1   5]
 [  2 140  15   3]
 [  1  12  13   1]
 [  0   4   0   6]]


**SUPPORT VECTOR MACHINE-GAUSSIAN RBF KERNEL**

In [17]:
#without weights
SVM_rbf = svm.SVC(kernel='rbf', C=250,gamma=0.05).fit(X_train, Y_train)
print("Accuracy:",SVM_rbf.score(X_test, Y_test))

Accuracy: 0.9166666666666666


In [18]:
CRandCM(SVM_rbf,X_test,Y_test)

              precision    recall  f1-score   support

           0       0.99      0.96      0.97       331
           1       0.87      0.93      0.89       160
           2       0.52      0.52      0.52        27
           3       0.56      0.50      0.53        10

    accuracy                           0.92       528
   macro avg       0.73      0.73      0.73       528
weighted avg       0.92      0.92      0.92       528

[[317   8   4   2]
 [  3 148   8   1]
 [  1  11  14   1]
 [  0   4   1   5]]


In [19]:
#weighted
SVM_rbf2= svm.SVC(kernel='rbf',C=250,gamma=0.01)
SVM_rbf2.fit(X_train, Y_train, sample_weight=weights)
print("Accuracy:",SVM_rbf2.score(X_test,Y_test))

Accuracy: 0.9299242424242424


In [20]:
CRandCM(SVM_rbf2,X_test,Y_test)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       331
           1       0.87      0.95      0.91       160
           2       0.63      0.44      0.52        27
           3       0.50      0.60      0.55        10

    accuracy                           0.93       528
   macro avg       0.75      0.74      0.74       528
weighted avg       0.93      0.93      0.93       528

[[321   5   2   3]
 [  1 152   5   2]
 [  1  13  12   1]
 [  0   4   0   6]]


**SUPPORT VECTOR MACHINE-SIGMOID KERNEL**

In [21]:
#without weights
SVM_sigmoid = svm.SVC(kernel='sigmoid', C=250,random_state=31).fit(X_train, Y_train)
print("Accuracy:",SVM_sigmoid.score(X_test, Y_test))

Accuracy: 0.11174242424242424


In [22]:
CRandCM(SVM_sigmoid,X_test,Y_test)

              precision    recall  f1-score   support

           0       0.70      0.10      0.17       331
           1       0.21      0.03      0.05       160
           2       0.07      0.52      0.13        27
           3       0.03      0.70      0.05        10

    accuracy                           0.11       528
   macro avg       0.25      0.34      0.10       528
weighted avg       0.51      0.11      0.13       528

[[ 33  17 117 164]
 [ 10   5  57  88]
 [  3   2  14   8]
 [  1   0   2   7]]


In [23]:
#weighted
SVM_sigmoid2= svm.SVC(kernel='sigmoid',C=250,gamma=0.01)
SVM_sigmoid2.fit(X_train, Y_train, sample_weight=weights)
print("Accuracy:",SVM_sigmoid2.score(X_test,Y_test))

Accuracy: 0.21022727272727273


In [24]:
CRandCM(SVM_sigmoid,X_test,Y_test)

              precision    recall  f1-score   support

           0       0.70      0.10      0.17       331
           1       0.21      0.03      0.05       160
           2       0.07      0.52      0.13        27
           3       0.03      0.70      0.05        10

    accuracy                           0.11       528
   macro avg       0.25      0.34      0.10       528
weighted avg       0.51      0.11      0.13       528

[[ 33  17 117 164]
 [ 10   5  57  88]
 [  3   2  14   8]
 [  1   0   2   7]]


**SUPPORT VECTOR MACHINE-POLYNOMIAL KERNEL**

In [25]:
#without weights
SVM_poly = svm.SVC(kernel ='poly', degree = 5, C=250).fit(X_train, Y_train)
print("Accuracy:",SVM_poly.score(X_test, Y_test))

Accuracy: 0.9223484848484849


In [26]:
CRandCM(SVM_poly,X_test,Y_test)

              precision    recall  f1-score   support

           0       0.99      0.96      0.98       331
           1       0.92      0.91      0.91       160
           2       0.63      0.63      0.63        27
           3       0.29      0.60      0.39        10

    accuracy                           0.92       528
   macro avg       0.71      0.77      0.73       528
weighted avg       0.94      0.92      0.93       528

[[319   3   2   7]
 [  2 145   7   6]
 [  1   7  17   2]
 [  1   2   1   6]]


In [27]:
#weighted
SVM_poly2= svm.SVC(kernel ='poly', degree = 5,C=250)
SVM_poly2.fit(X_train, Y_train, sample_weight=weights)
print("Accuracy:",SVM_poly2.score(X_test,Y_test))

Accuracy: 0.9242424242424242


In [28]:
CRandCM(SVM_poly2,X_test,Y_test)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       331
           1       0.89      0.93      0.91       160
           2       0.56      0.52      0.54        27
           3       0.46      0.60      0.52        10

    accuracy                           0.92       528
   macro avg       0.73      0.75      0.74       528
weighted avg       0.93      0.92      0.93       528

[[320   5   2   4]
 [  2 148   8   2]
 [  1  11  14   1]
 [  1   2   1   6]]


RBF kernel and polynomial kernel are to better kernels